# CSE 258, Fall 2021: Homework 3

## Tasks (Cook/Make prediction)

In [8]:
import gzip
import csv
import random
import pandas as pd
from collections import defaultdict
from sklearn import linear_model
from tqdm.notebook import tqdm

In [2]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [3]:
def readCSV(path):
    f = gzip.open(path, 'rt')
    c = csv.reader(f)
    header = next(c)
    for l in c:
        d = dict(zip(header,l))
        yield d['user_id'],d['recipe_id'],d

In [4]:
def splitDataset(path):
    f = gzip.open(path, 'rt')
    data = pd.read_csv(f)
    train, valid = data[:400000], data[400000:]
    return data, train, valid

In [5]:
data, train, valid = splitDataset("trainInteractions.csv.gz")

### Q1
Although we have built a validation set, it only consists of positive samples. For this task we also need examples of user/item pairs corresponding to recipes that weren’t cooked. For each entry (user,recipe) in the validation set, sample a negative entry by randomly choosing a recipe that user hasn’t cooked. Evaluate the performance (accuracy) of the baseline model on the validation set you have built.

In [9]:
def sampleNegative(data, train, valid):
    valid['cooked'] = 1
    NegValid = valid

    userRecipeDict = {}
    for index, row in tqdm(data.iterrows()):
        if row['user_id'] not in userRecipeDict:
            userRecipeDict[row['user_id']] = {row['recipe_id']}
        else:
            userRecipeDict[row['user_id']].add(row['recipe_id'])

    for index, row in tqdm(valid.iterrows()):
        NegRecipeID = random.sample(set(data['recipe_id']) - userRecipeDict[row['user_id']], 1)[0]
        NegValid = NegValid.append({'user_id': row['user_id'], 'recipe_id': NegRecipeID, 'rating': 0, 'cooked':0}, ignore_index=True)

    return NegValid, userRecipeDict

In [10]:
valid, _ = sampleNegative(data, train, valid)

<ipython-input-9-d8cac14a1116>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['cooked'] = 1


0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [ ]:
def Q1():    
    recipeCount = defaultdict(int)
    totalCooked = 0

    for index, row in tqdm(train.iterrows()):
        recipeCount[row['recipe_id']] += 1
        totalCooked += 1

    mostPopular = [(recipeCount[x], x) for x in recipeCount]
    mostPopular.sort()
    mostPopular.reverse()

    return1 = set()
    count = 0
    for ic, i in mostPopular:
        count += ic
        return1.add(i)
        if count > totalRead / 2:
            break

    correct = 0
    for index, row in tqdm(valid.iterrows()):
        if row['recipe_id'] in return1:
            correct += (row['cooked'] != 0)
        else:
            correct += (row['cooked'] == 0)
    
    print('Accuracy on Validation set is ' % (correct/len(valid)))

In [ ]:
Q1()

### Q2
The existing ‘made/cooked prediction’ baseline just returns True if the item in question is ‘popular,’ using a threshold of the 50th percentile of popularity (totalCooked/2). Assuming that the ‘non-made’ test examples are a random sample of user-recipe pairs, this threshold may not be the best one. See if you can find a better threshold and report its performance on your validation set.

In [ ]:
def Q2():
    recipeCount = defaultdict(int)
    totalCooked = 0
    
    threshold = 1.5

    for index, row in tqdm(train.iterrows()):
        recipeCount[row['recipe_id']] += 1
        totalCooked += 1

    mostPopular = [(recipeCount[x], x) for x in recipeCount]
    mostPopular.sort()
    mostPopular.reverse()

    return1 = set()
    count = 0
    for ic, i in mostPopular:
        count += ic
        return1.add(i)
        if count > totalRead / threshold:
            break

    correct = 0
    for index, row in tqdm(valid.iterrows()):
        if row['recipe_id'] in return1:
            correct += (row['cooked'] != 0)
        else:
            correct += (row['cooked'] == 0)
    
    print('Accuracy on Validation set is ' % (correct/len(valid)))

In [ ]:
Q2()

### Q3
An alternate baseline than the one provided might make use of the Jaccard similarity (or another similarity metric). Given a pair (u, g) in the validation set, consider all training items g0 that user u has cooked. For each, compute the Jaccard similarity between g and g0, i.e., users (in the training set) who have made g and users who have made g0. Predict as ‘made’ if the maximum of these Jaccard similarities exceeds a threshold (you may choose the threshold that works best). Report the performance on your validation set.

In [ ]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    return numer / denom

In [ ]:
userReadBook, BookwasRead = {}, {}
for index, row in tqdm(train.iterrows()):
    if row['userID'] not in userReadBook:
        userReadBook[row['userID']] = {row['bookID']}
    else:
        userReadBook[row['userID']].add(row['bookID'])
    if row['bookID'] not in BookwasRead:
        BookwasRead[row['bookID']] = {row['userID']}
    else:
        BookwasRead[row['bookID']].add(row['userID'])

In [ ]:
thresholds = [1/2**i for i in range(1, 10)]
acc = []
for threshold in thresholds:
    print('Evaluating on threshold %.3f ...' % threshold)
    correct = 0
    for index, row in tqdm(valid.iterrows()):
        userReads = userReadBook[row['userID']]
        jac = []
        for book in userReads:
            if row['bookID'] not in BookwasRead:
                jac.append(0)
            else:
                jac.append(Jaccard(BookwasRead[row['bookID']], BookwasRead[book]))
                
        if max(jac) > threshold:
            correct += (row['read'] != 0)
        else:
            correct += (row['read'] == 0)
    
    acc.append(correct/len(valid))

plt.plot(thresholds, acc, 'b-')
plt.xlabel('Threshold')
plt.ylabel('Accuracy on Validation Set for different thresholds')
plt.show()

In [ ]:
print('threshold %.3f, accuracy %.3f' % (thresholds[acc.index(max(acc))], max(acc)))

In [1]:
import tensorflow